In [1]:
from os import path
import pandas as pd
from glob import glob
from util.table_converter import TableConverter, TableProps;
from generator.constant import Constant

In [2]:
target_project_names = list(map(lambda s : s.split('/')[-1].split('.')[0], glob('../compile_information/*.json')))
target_project_names.remove('template')
target_project_names.remove('compile_information')

target_project_root_paths = [Constant.local_project_root_path + '/' + tpn for tpn in target_project_names]

project_root_path = Constant.output_root_path + '/' + 'project.csv'
project_df = pd.read_csv(project_root_path)

file_root_path = Constant.output_root_path + '/' + 'file.csv'
file_df = pd.read_csv(file_root_path)

available_macro_root_path = Constant.output_root_path + '/' + 'available_macros.csv'
available_macro_df = pd.read_csv(available_macro_root_path)

used_macro_root_path = Constant.output_root_path + '/' + 'used_macros.csv'
used_macro_df = pd.read_csv(used_macro_root_path)

defined_macro_root_path = Constant.output_root_path + '/' + 'defined_macros.csv'
defined_macro_df = pd.read_csv(defined_macro_root_path)

whole_macro_root_path = Constant.output_root_path + '/' + 'whole_macros.csv'
whole_macro_df = pd.read_csv(whole_macro_root_path)

macros_root_path =Constant.output_root_path + '/' + 'macros.csv'
macros_df = pd.read_csv(macros_root_path)

tvc = TableConverter()


In [12]:
def calc_define_directive_rate():
    columns = pd.Series(['project name', 'file num', 'target file num', 'rate'])
    macro_per_files_count = defined_macro_df.groupby(['file_id']).count().sort_values(['macro_id'], ascending=False)
    file_macro_count_df = pd.merge(macro_per_files_count, file_df, on='file_id', how='left')
    file_macro_df = pd.merge(defined_macro_df, file_df, on='file_id', how='left')
    del file_macro_count_df['name']
    del file_macro_count_df['path']
    del file_macro_count_df['id']
    del file_macro_df['name']
    del file_macro_df['path']
    del file_macro_df['id']
    target_p_names = project_df[['project_id', 'name']]
    rows = []
    for _, dt in target_p_names.iterrows():
        if not dt['name'] in target_project_names:
            continue
        row = [dt['name']]
        filterd_file_df = file_df[file_df['project_id'] == dt['project_id']]
        target_filterd_file_df = file_macro_count_df[file_macro_count_df['project_id'] == dt['project_id']]
        row += [
            len(filterd_file_df),
            len(target_filterd_file_df),
            round(len(target_filterd_file_df) / len(filterd_file_df), 4)
        ]
        rows.append(row)
    return pd.DataFrame(rows, columns=columns)

def calc_used_macro_rate():
    columns = pd.Series(['project name', 'file num', 'target file num', 'rate'])
    macro_per_files_count = used_macro_df.groupby(['file_id']).count().sort_values(['macro_id'], ascending=False)
    file_macro_count_df = pd.merge(macro_per_files_count, file_df, on='file_id', how='left')
    file_macro_df = pd.merge(used_macro_df, file_df, on='file_id', how='left')
    del file_macro_count_df['name']
    del file_macro_count_df['path']
    del file_macro_count_df['id']
    del file_macro_df['name']
    del file_macro_df['path']
    del file_macro_df['id']
    target_p_names = project_df[['project_id', 'name']]
    rows = []
    for _, dt in target_p_names.iterrows():
        if not dt['name'] in target_project_names:
            continue
        row = [dt['name']]
        filterd_file_df = file_df[file_df['project_id'] == dt['project_id']]
        target_filterd_file_df = file_macro_count_df[file_macro_count_df['project_id'] == dt['project_id']]
        row += [
            len(filterd_file_df),
            len(target_filterd_file_df),
            round(len(target_filterd_file_df) / len(filterd_file_df), 4)
        ]
        rows.append(row)
    return pd.DataFrame(rows, columns=columns)

## Define Macro rate of each files

In [16]:
target_df = calc_define_directive_rate()
tvc.save_as_table(TableProps(title='appearance of define directive in each project', file_name='appearance_of_define_directive_in_each_project', data=target_df))
print(target_df)

  project name  file num  target file num    rate
0      radare2      1888              957  0.5069
1        nginx       343              178  0.5190
2     goaccess        70               19  0.2714
3          nnn        10                9  0.9000
4          ish       216              116  0.5370
5       brotli        93               69  0.7419


## Used Macro rate of each files

In [17]:
target_df = calc_used_macro_rate()
tvc.save_as_table(TableProps(title='frequcncy of macro use file in each project', file_name='frequency_of_macro_use_file_in_each_project', data=target_df))
print(target_df)

  project name  file num  target file num    rate
0      radare2      1888             1339  0.7092
1        nginx       343              271  0.7901
2     goaccess        70               30  0.4286
3          nnn        10                3  0.3000
4          ish       216              121  0.5602
5       brotli        93               62  0.6667
